In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import smtplib as smtp
from getpass import getpass

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("glaiveai/glaive-function-calling-v1", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("glaiveai/glaive-function-calling-v1", trust_remote_code=True).half().cuda()

In [ ]:
def send_email(destination_email: str, message_text: str):
    """Send message to user_email with message_text"""

    email = ''
    password = ''
    dest_email = destination_email
    subject = 'Test'
    email_text = message_text

    message = 'From: {}\nTo: {}\nSubject: {}\n\n{}'.format(email,
                                                        dest_email, 
                                                        subject, 
                                                        email_text)

    server = smtp.SMTP_SSL('smtp.yandex.com')
    server.set_debuglevel(1)
    server.ehlo(email)
    server.login(email, password)
    server.auth_plain()
    server.sendmail(email, dest_email, message)
    server.quit()

In [ ]:
prompt = """
SYSTEM: You are an helpful assistant who has access to the following functions to help the user, you can use the functions if needed-
{
            "name": "send_email",
            "description": "Plan a holiday based on user's interests",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_email": {
                        "type": "string",
                        "description": "Destination email",
                    },
                    "message_text": {
                        "type": "string",
                        "description": "Text of message",
                    },
                },
                "required": ["destination_email", "message_text"],
            },
}
USER: Send a message to email about the meeting on June 13. And tell him to call my number back. Be kind

"""

In [ ]:
def function_calling(prompt):
    inputs = tokenizer(prompt,return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs,do_sample=True,temperature=0.1,top_p=0.95,max_new_tokens=200)
    res = tokenizer.decode(outputs[0],skip_special_tokens=True)
    res = res.split("<functioncall>")[-1]
    res = json.loads(res)
    locals()[res["name"]](**res["arguments"])

function_calling(prompt)